In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import csv
import os
import numpy as np
import sys
from sklearn.model_selection import train_test_split
import importlib

# Import constants and catch any import errors
from link_to_constants import *
link_constants()
try:
    from constants import * # type: ignore
    print("Successfully imported constants.")
except ImportError as e:
    print(f"Error importing constants: {e}")

Current directory: c:\Users\marco\Documents\GitHub\Tesi\Codice_carla\final - Copy\neural_network
Parent directory: c:\Users\marco\Documents\GitHub\Tesi\Codice_carla\final - Copy
Data gen and processing directory: c:\Users\marco\Documents\GitHub\Tesi\Codice_carla\final - Copy\data_gen_and_processing
Successfully imported constants.


In [2]:
def load_points_grid_map(csv_file):
    """Load bounding box vertices from a CSV file."""
    points = []
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            # Extract the 3D coordinates of the 8 bounding box vertices
            coordinates = [
                float(row[0]), float(row[1]), float(row[2])
                ]
            points.append(coordinates)
    np_points = np.array(points)
    return np_points

def generate_grid_map (grid_map_path):
    grid_map_files = [f for f in os.listdir(grid_map_path)]
    list_grid_maps = []

    for file in grid_map_files:
        complete_path = os.path.join(grid_map_path, file)
        print(f"Loading {file}...")
        points = load_points_grid_map(complete_path)

        # Recreate the grid map from positions array
        grid_map_recreate = np.full((Y_RANGE, X_RANGE), FLOOR_HEIGHT, dtype=float) # type: ignore

        # Fill the grid map with values from positions array
        for pos in points:
            col, row, height = pos
            grid_map_recreate[int(row), int(col)] = height

        list_grid_maps.append(grid_map_recreate)
    
    return list_grid_maps

def load_points_grid_map_BB (csv_file):
    """Load bounding box vertices from a CSV file."""
    points = []
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header
        
        for row in reader:
            # Extract the 3D coordinates of the 8 bounding box vertices
            coordinates = [ [ float(row[i]), float(row[i+1]), float(row[i+2]) ] for i in range(2, 12, 3)]
            points.append(coordinates)

    np_points = np.array(points)
    return np_points

def generate_grid_map_BB (grid_map_path):
    grid_map_files = [f for f in os.listdir(grid_map_path)]
    list_grid_maps = []

    for file in grid_map_files:
        complete_path = os.path.join(grid_map_path, file)
        print(f"Loading {file}...")
        points = load_points_grid_map_BB(complete_path)

        # Recreate the grid map from positions array
        grid_map_recreate = np.full((Y_RANGE, X_RANGE), FLOOR_HEIGHT, dtype=float) # type: ignore

        # Fill the grid map with values from positions array
        for i in range (len(points)):
            for j in range(4):
                col, row, height = points[i][j]
                grid_map_recreate[int(row), int(col)] = height

        list_grid_maps.append(grid_map_recreate)
    
    return list_grid_maps

def split_data(lidar_data, BB_data, size):
    # Split the dataset into a combined training and validation set, and a separate test set
    X_train_val, X_test, y_train_val, y_test = train_test_split(
        lidar_data, # Samples
        BB_data, # Labels
        test_size = size,
        random_state=SEED # type: ignore
    )
    return X_train_val, X_test, y_train_val, y_test

In [3]:
complete_grid_maps = []
complete_grid_maps_BB = []

# Load sensor1
grid_maps = generate_grid_map(LIDAR_1_GRID_DIRECTORY) # type: ignore
grid_maps_BB = generate_grid_map_BB(NEW_POSITIONS_LIDAR_1_GRID_DIRECTORY) # type: ignore

complete_grid_maps.append(grid_maps)
complete_grid_maps_BB.append(grid_maps_BB)

# Load sensor2
grid_maps = generate_grid_map(LIDAR_2_GRID_DIRECTORY) # type: ignore
grid_maps_BB = generate_grid_map_BB(NEW_POSITIONS_LIDAR_2_GRID_DIRECTORY) # type: ignore

complete_grid_maps.append(grid_maps)
complete_grid_maps_BB.append(grid_maps_BB)

# Load sensor3
grid_maps = generate_grid_map(LIDAR_3_GRID_DIRECTORY) # type: ignore
grid_maps_BB = generate_grid_map_BB(NEW_POSITIONS_LIDAR_3_GRID_DIRECTORY) # type: ignore

complete_grid_maps.append(grid_maps)
complete_grid_maps_BB.append(grid_maps_BB)

Loading 20250118_102725_853040_169399.csv...
Loading 20250118_102725_975855_169402.csv...
Loading 20250118_102726_071063_169404.csv...
Loading 20250118_102726_162930_169406.csv...
Loading 20250118_102726_243632_169408.csv...
Loading 20250118_102726_321813_169410.csv...
Loading 20250118_102726_399399_169412.csv...
Loading 20250118_102726_476882_169414.csv...
Loading 20250118_102726_558496_169416.csv...
Loading 20250118_102726_635325_169418.csv...
Loading 20250118_102726_713490_169420.csv...
Loading 20250118_102726_789580_169422.csv...
Loading 20250118_102726_868563_169424.csv...
Loading 20250118_102726_947714_169426.csv...
Loading 20250118_102727_022729_169428.csv...
Loading 20250118_102727_100899_169430.csv...
Loading 20250118_102727_181769_169432.csv...
Loading 20250118_102727_260511_169434.csv...
Loading 20250118_102727_348745_169436.csv...
Loading 20250118_102727_432632_169438.csv...
Loading 20250118_102727_522459_169440.csv...
Loading 20250118_102727_614992_169442.csv...
Loading 20

In [4]:
print(len(complete_grid_maps[0]), len(complete_grid_maps[1]), len(complete_grid_maps[2]))

284 367 317


In [5]:
print(len(complete_grid_maps_BB[0]), len(complete_grid_maps_BB[1]), len(complete_grid_maps_BB[2]))

284 367 317


In [6]:
# Concatenate the lists in complete_grid_maps along the first dimension
complete_grid_maps_np = np.concatenate(complete_grid_maps, axis=0)
print(complete_grid_maps_np.shape)

(968, 400, 400)


In [7]:
# Concatenate the lists in complete_grid_maps_BB along the first dimension
complete_grid_maps_BB_np = np.concatenate(complete_grid_maps_BB, axis=0)
print(complete_grid_maps_BB_np.shape)

(968, 400, 400)


In [8]:
print(complete_grid_maps_np[0][200][200])

-58.94


In [9]:
# Split the data
X_train_val, X_test, y_train_val, y_test = split_data(complete_grid_maps_np, complete_grid_maps_BB_np, TEST_SIZE) # type: ignore
X_train, X_val, y_train, y_val = split_data(X_train_val, y_train_val, len(X_test)) # Esure that val and test set have the same lenght

In [10]:
print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)

(774, 400, 400) (97, 400, 400) (97, 400, 400) (774, 400, 400) (97, 400, 400) (97, 400, 400)


In [11]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the scaler
scaler = MinMaxScaler()

# Normalize the data
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_val = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

y_train = scaler.fit_transform(y_train.reshape(-1, y_train.shape[-1])).reshape(y_train.shape)
y_val = scaler.transform(y_val.reshape(-1, y_val.shape[-1])).reshape(y_val.shape)
y_test = scaler.transform(y_test.reshape(-1, y_test.shape[-1])).reshape(y_test.shape)

In [12]:
print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)

(774, 400, 400) (97, 400, 400) (97, 400, 400) (774, 400, 400) (97, 400, 400) (97, 400, 400)


In [13]:
X_train = np.expand_dims(X_train, axis=1)
X_val = np.expand_dims(X_val, axis=1)
X_test = np.expand_dims(X_test, axis=1)
print("New input shape (train, val, test):", X_train.shape, X_val.shape, X_test.shape)

New input shape (train, val, test): (774, 1, 400, 400) (97, 1, 400, 400) (97, 1, 400, 400)


In [14]:
y_train = np.expand_dims(y_train, axis=1)
y_val = np.expand_dims(y_val, axis=1)
y_test = np.expand_dims(y_test, axis=1)
print("New labels shape (train, val, test):", y_train.shape, y_val.shape, y_test.shape)

New labels shape (train, val, test): (774, 1, 400, 400) (97, 1, 400, 400) (97, 1, 400, 400)


In [15]:
# Define the autoencoder model
class Autoencoder(nn.Module):
    def __init__(self): # Constructor method for the autoencoder
        super(Autoencoder, self).__init__() # Calls the constructor of the parent class (nn.Module) to set up necessary functionality.
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride = 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride = 2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride = 2)
        )
        self.decoder = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(32, 1, kernel_size=3, padding=1),
        )

    def forward(self, x): # The forward method defines the computation that happens when the model is called with input x.
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [16]:
# Prepare data loaders
train_dataset = TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float()) # Each element in train_dataset will be a tuple (input, target). Both will have shape (400,400). There will be as many elements in the dataset as there are samples in X_train.
val_dataset = TensorDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).float())
test_dataset = TensorDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).float())

In [17]:
print(len(train_dataset), len(val_dataset), len(test_dataset))

774 97 97


In [18]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
model = Autoencoder()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001) # model.parameters() passes the parameters of the model to the optimizer so that it can update them during training.

In [ ]:
# Check if CUDA is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    model = model.to(device)
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

In [19]:
# Clear GPU cache
torch.cuda.empty_cache()

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for data in train_loader:
        inputs, targets = data
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad() # In PyTorch, gradients are accumulated by default. This means that when you call loss.backward(), the gradients computed for each parameter are added to any existing gradients stored in those parameters. You want to reset the gradients at the start of each training step to avoid mixing gradient information from different batches.
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader) # Average loss over all batches

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for data in val_loader:
            inputs, targets = data
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()
    val_loss /= len(val_loader)

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    # Clear GPU cache
    torch.cuda.empty_cache()

# Evaluate on test set
model.eval()
test_loss = 0
with torch.no_grad():
    for data in test_loader:
        inputs, targets = data
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()
test_loss /= len(test_loader)
print(f'Test Loss: {test_loss:.4f}')
# Clear GPU cache
torch.cuda.empty_cache()


# Make predictions
predictions = []
with torch.no_grad():
    for data in test_loader:
        inputs, _ = data
        inputs= inputs.to(device)
        outputs = model(inputs)
        predictions.append(outputs)
predictions = torch.cat(predictions).cpu().numpy()
print("Predictions Shape:", predictions.shape)
# Clear GPU cache
torch.cuda.empty_cache()


Using GPU: NVIDIA GeForce RTX 4050 Laptop GPU
Epoch 1/20, Train Loss: 0.0001, Val Loss: 0.0018
Epoch 2/20, Train Loss: 0.0000, Val Loss: 0.0018
Epoch 3/20, Train Loss: 0.0000, Val Loss: 0.0018
Epoch 4/20, Train Loss: 0.0000, Val Loss: 0.0018


KeyboardInterrupt: 

In [12]:
value_batch_size = 128   # Batch size for training
value_epochs = 5
value_learning_rate = 1e-3
value_callbacks=[
    tfk.callbacks.EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True, mode='max'),
    tfk.callbacks.ReduceLROnPlateau(monitor="val_accuracy", factor=0.1, patience=15, min_lr=1e-5, mode='max')
]

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

In [ ]:
history = model.fit(
    x = X_train, 
    y = y_train,
    batch_size = value_batch_size,
    epochs = value_epochs,
    validation_data = (X_val,y_val),
    verbose=1,
    callbacks = value_callbacks
)

In [ ]:
predictions = model.predict(X_test, verbose=0)

# Display the shape of the predictions
print("Predictions Shape:", predictions.shape)

predictions